## Softmax Classification

- 로지스틱 회귀분석의 연장선
- 순서
    1. Softmax
    2. Cross Entropy
    3. Low-level Implementation
    4. High-level Implementation
    5. Training Example

### Import

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#For reproductibility
torch.manual_seed(1)

### Discrete Probability Distribution
- 이산적인 확률 분포

### Softmax
- 부드럽게 max를 뽑아준다는 뜻.
- max(1,2,3)=(0,0,1)
- softmax(1,2,3)=(0.09,0.24,0.66) 이런 식으로 확률을 뽑아준다.
- 0.09=e^1/(e^1+e^2+e^3) 이렇게 계산한다.

In [3]:
z=torch.FloatTensor([1,2,3])

hypothesis=F.softmax(z,dim=0)
print(hypothesis)
hypothesis.sum()

tensor([0.0900, 0.2447, 0.6652])


tensor(1.)

### Cross Entropy
- 두 개의 확률분포가 주어졌을 때, 두 확률분포가 얼마나 비슷한지 나타낼 수 있는 수치
- 원래 확률 P와 추정 확률 Q가 있다.
- P,Q의 cross entropy가 작을수록 원래 확률 P에 가까워진다.
- 따라서 cross entropy를 minimize하는 것이 중요하다.
- cross entropy loss 계산하기

In [4]:
z=torch.rand(3,5,requires_grad=True)
hypothesis=F.softmax(z,dim=1)
#dim=1 이면 각 행에 대해 softmax를 수행하라는 뜻.
print(hypothesis)
# classes가 5개,  samples가 3개

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [6]:
y=torch.randint(5,(3,)).long()
print(y) 
#각각 인덱스에 대해 랜덤으로 만들어낸 정답

tensor([2, 1, 0])


- 정답이 [2,1,0] 이므로 [0.1855,0.1624,0.2226] 이 될 것이다.
- 이게 무슨 소린지 아직 잘 모르겠다.
- 우리는 이것을 one-hot vector로 표현할 수 있다.
- one-hot vector는 데이터를 0,1로 이루어진 벡터로 변환하는 것. [1000], [0100] 이런 식이다/ 

In [9]:
y_one_hot=torch.zeros_like(hypothesis)
y_one_hot.scatter_(1,y.unsqueeze(1),1)
# 언더바는 in place 함수라서 y_one_hot 자체로 바뀌었음.
# (2,1,0)>>(2,1,0)^t>>[00100][01000][10000] 으로 변환

tensor([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [10]:
cost=(y_one_hot* -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)
# 아까 cross entropy 수식
# (3,5)>>>(3,)>>>scalar 이 순서로 변형

tensor(1.6682, grad_fn=<MeanBackward1>)


- 방금 수행한 부분은 조금 더 쉽게 할 수 있다.
- 파이토치에서 함수를 제공한다.
- `torch.nn.functional`

In [11]:
#Low level
torch.log(F.softmax(z,dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [13]:
# High level
F.log_softmax(z,dim=1)
# 위에랑 같은 내용이 나옴.

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [14]:
#low level
(y_one_hot* -torch.log(hypothesis)).sum(dim=1).mean()

tensor(1.6682, grad_fn=<MeanBackward1>)

In [15]:
#High level
F.nll_loss(F.log_softmax(z,dim=1),y)
#NLL 은 Negative Log Likelihood loss의 줄임말

tensor(1.6682, grad_fn=<NllLossBackward>)

In [18]:
F.cross_entropy(z,y) 
#진짜 제일 간단하게 하는 방법
#근데 이건 한번에 해버리기 때문에 나중에 softmax값이 필요할 경우에는 쓸 수 없음
#따라서 각자에게 맞는걸로 선택하면 된다.

tensor(1.6682, grad_fn=<NllLossBackward>)

이제 직접 예시에 대입해보자!

### low level cross entropy

In [19]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
#x_train (m,4) m은 샘플 갯수
#y_train (m,)
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [20]:
#모델 초기화
# sample m, class 3, dim 4, 그래서 4>3으로 가는 linear layer 만들었다.
W=torch.zeros((4,3),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

#optimizer 설정
optimizer=optim.SGD([W,b],lr=0.1)

nb_epochs=1000
for epoch in range(nb_epochs+1):
    
    #Cost 계산 (1)
    hypothesis=F.softmax(x_train.matmul(W)+b,dim=1)
    y_one_hot=torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
    cost=(y_one_hot* -torch.log(hypothesis)).sum(dim=1).mean()
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs,cost.item()))
        

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### `F.cross_entropy` 를 이용하자

In [21]:
#모델 초기화
# sample m, class 3, dim 4, 그래서 4>3으로 가는 linear layer 만들었다.
W=torch.zeros((4,3),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

#optimizer 설정
optimizer=optim.SGD([W,b],lr=0.1)

nb_epochs=1000
for epoch in range(nb_epochs+1):
    
    #Cost 계산 (2) 아까랑 여기만 달라짐
    z=x_train.matmul(W)+b
    cost=F.cross_entropy(z,y_train)
    
    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs,cost.item()))
      

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


### High-level Implementation with `nn.Module`

In [24]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=nn.Linear(4,3) #output이 3이다!
        
    def forward(self,x):
        return self.linear(x)

In [25]:
model=SoftmaxClassifierModel()

In [26]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산 여기가 한 줄로 정리됨.
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 4.661125
Epoch  100/1000 Cost: 0.711834
Epoch  200/1000 Cost: 0.630354
Epoch  300/1000 Cost: 0.573769
Epoch  400/1000 Cost: 0.523962
Epoch  500/1000 Cost: 0.476843
Epoch  600/1000 Cost: 0.430889
Epoch  700/1000 Cost: 0.385375
Epoch  800/1000 Cost: 0.339987
Epoch  900/1000 Cost: 0.295069
Epoch 1000/1000 Cost: 0.254579


- class가 2개인 경우
    - Binary cross entropy, sigmoid 함수
- class가 3개 이상인 경우
    - cross entropy, softmax 함수